# Working with Excel using the ImportExcel Module

The ImportExcel Module was started by Doug Finke [b](https://dfinke.github.io/) [t](https://twitter.com/dfinke) and has consistently been updated with significant input from James O'Neill [b](https://jamesone111.wordpress.com/) [t](https://twitter.com/jamesoneill) amongst others. ImportExcel enables automation of Excel workbooks using PowerShell and works with both Windows PowerShell and PowerShell Core

## Installation
Lets start by installing it (we will also check that you have dbatools too!)

In [ ]:
$Modules = 'ImportExcel','dbatools','Microsoft.PowerShell.SecretManagement','Microsoft.PowerShell.SecretStore'
foreach($Module in $Modules){
    if(Get-Module $Module -ListAvailable -ErrorAction SilentlyContinue){
        Write-Output "We will update the module $Module"
        Update-Module $Module
    } else {
        Write-Output "We will install the module $Module"
        Install-Module $Module -Scope CurrentUser
    }
}


We will update the module ImportExcel


We will update the module dbatools
We will update the module Microsoft.PowerShell.SecretManagement
We will update the module Microsoft.PowerShell.SecretStore
Update-Module: 
Line |
   5 |          Update-Module $Module
     |          ~~~~~~~~~~~~~~~~~~~~~
     | Module 'Microsoft.PowerShell.SecretStore' was not installed by using Install-Module, so it cannot be updated.


## Listing the commands

The module has a significant number of commands

In [ ]:
## first we will set some values that will be used throughout the notebook
$dbachecks1 = 'localhost,7401' # unless we are inside the container Rob
$dbachecks2 = 'localhost,7402' # unless we are inside the container Rob
$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name sqladmin -AsPlainText), (Get-Secret -Name sqladminpwd))


$PSDefaultParameterValues = @{
    "*dba*:SqlInstance" = $dbachecks1 
    "*dba*:SqlCredential" = $benscreds 
}

$sql

In [ ]:
Get-Command -Module ImportExcel


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Alias           Convert-XlRangeToImage                             7.8.1      ImportExcel
Alias           Export-ExcelSheet                                  7.8.1      ImportExcel
Alias           New-ExcelChart                                     7.8.1      ImportExcel
Alias           Set-Column                                         7.8.1      ImportExcel
Alias           Set-Format                                         7.8.1      ImportExcel
Alias           Set-Row                                            7.8.1      ImportExcel
Alias           Use-ExcelData                                      7.8.1      ImportExcel
Function        Add-ConditionalFormatting                          7.8.1      ImportExcel
Function        Add-ExcelChart                                     7.8.1      ImportExcel
Function        Add

## Start with something easy

Let's start of by showing the power of PowerShell and the ImportExcel module

First lets get the logins on an instance with dbatools, you can list them here and save the results in a notebook or you can copy the results and paste them into an email or somthing else, maybe Excel!

In [ ]:
## first we will set some values that will be used throughout the notebook
$dbachecks1 = 'localhost,7401' # unless we are inside the container Rob
$dbachecks2 = 'localhost,7402' # unless we are inside the container Rob
$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name sqladmin -AsPlainText), (Get-Secret -Name sqladminpwd))


$PSDefaultParameterValues = @{
    "*dba*:SqlInstance" = $dbachecks1 
    "*dba*:SqlCredential" = $benscreds 
}

$sql

In [ ]:
Get-DbaLogin


ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : mssql1
Name               : ##MS_PolicyEventProcessingLogin##
LoginType          : SqlLogin
CreateDate         : 12/01/2022 22:54:29
LastLogin          : 
HasAccess          : True
IsLocked           : False
IsDisabled         : True
MustChangePassword : False

ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : mssql1
Name               : ##MS_PolicyTsqlExecutionLogin##
LoginType          : SqlLogin
CreateDate         : 12/01/2022 22:54:29
LastLogin          : 
HasAccess          : True
IsLocked           : False
IsDisabled         : True
MustChangePassword : False

ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : mssql1
Name               : akamman
LoginType          : SqlLogin
CreateDate         : 06/10/2022 13:23:14
LastLogin          : 
HasAccess          : True
IsLocked           : False
IsDisabled         : False
MustChangePas

Of course, your user will want them in Excel!!

With ImportExcel Module, it is easy to pipe the results from any command into an excel worksheet

In [ ]:
Get-DbaLogin  | Export-Excel -Path C:\temp\xlsx\Logins.xlsx
Invoke-Item C:\temp\xlsx\Logins.xlsx

Thats cool, <u>but we want more</u>.

We will often autosize the results in Excel so that the columns fit. It would be good to be able to auto format with PowerShell. With the ImportExcel you can do that with the `AutoSize` parameter

In [ ]:
Get-DbaLogin | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize
Invoke-Item C:\temp\xlsx\Logins.xlsx

 Thats neat but I want to be able to filter by the columns as well. You can use the `-AutoFilter` parameter for this

In [ ]:
Get-DbaLogin | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize -AutoFilter
Invoke-Item C:\temp\xlsx\Logins.xlsx

To make it look a little nicer, I could name the worksheet and give the table a title.
I want a title and to be able to name the worksheet and because I am lazy, I want it to just open with one line.

You can use the `-Title`, `-WorksheetName`, and `-Show` parameters to achieve this.

In [ ]:
Get-DbaLogin | Export-Excel -Path C:\temp\xlsx\Logins.xlsx -AutoSize -AutoFilter -Title "The Logins" -WorksheetName "Logins" -Show 

**Hang on a minute, what happened there?**

It has created a new worksheet and added it onto the existing workbook but it made it the second one!

Lets fix that with the `MoveToStart` parameter, We'll make the background yellow so its obvious and used the `FreezeTopRow` parameter to, well freeze the top row. I have also changed the code to use `splatting` which I think makes it easier to read.

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\Logins.xlsx' 
    AutoSize =  $true
    AutoFilter =  $true
    Title =  "The Logins" 
    WorksheetName =  "Logins" 
    MoveToStart =  $true
    TitleBackgroundColor =  'yellow' 
    FreezeTopRow =  $true
    Show =  $true
}
Get-DbaLogin | Export-Excel @ExportExcelParams

# Use cases

This is really neat, its a simple way of getting the results of any PowerShell command, quickly and easily into an Excel sheet for sorting.

At the command line, you can make sure that you are getting the results that you need and then pipe those results into the Export-Excel command.

## Services

So you could get the services like this.

`FreezeTopRow` freezes the top row which is the title! We need to freeze the second row. To do this you use the `FreezePane` parameter and provide the rownumber and column number. In this case row 3 and column 0.

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\Services.xlsx' 
    AutoSize =  $true
    AutoFilter =  $true
    Title =  "Services on $Env:ComputerName" 
    WorksheetName =  "Services" 
    MoveToStart =  $true
    TitleBackgroundColor =  'darkgray' 
    FreezePane = 3,0
    Show =  $true
}
Get-Service| Export-Excel @ExportExcelParams

## Processes

Or the Processes like this. This takes a little minute

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\Processes.xlsx' 
    AutoSize =  $true
    AutoFilter =  $true
    Title =  "Processes on $Env:ComputerName" 
    WorksheetName =  "Processes" 
    MoveToStart =  $true
    TitleBackgroundColor =  'darkgray' 
    FreezePane = 3,0
    Show =  $true
}
Get-Process | Export-Excel @ExportExcelParams

## Event Logs

You can do the same thing with Windows Event logs. This time you will add both Application and System Event Log entries into seperate worksheets

In [ ]:
$ExportExcelParams = @{
    Path = 'C:\temp\xlsx\EventLogs.xlsx'
    AutoSize =  $true
    AutoFilter =  $true
    Title =  "Event Log on $Env:ComputerName" 
    MoveToStart =  $true
    TitleBackgroundColor =  'darkgray' 
    FreezePane = 3,0
}
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel @ExportExcelParams  -WorksheetName "System" 
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel @ExportExcelParams  -WorksheetName "Application" -Show

###  SQL Error Logs

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\SQLErrorLogs.xlsx' 
    AutoSize =  $true
    AutoFilter =  $true
    Title =  "SQL Error Log" 
    WorksheetName =  "SQL Error Log" 
    MoveToStart =  $true
    TitleBackgroundColor =  'darkgray' 
    FreezePane = 3,0
    Show =  $true
}
Get-DbaErrorLog  | Export-Excel  @ExportExcelParams


## How About doing it all in one workbook ?

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\Incident.xlsx' 
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
}

Get-Service | Export-Excel @ExportExcelParams -WorksheetName "Services" 
# Get-Process | Export-Excel @ExportExcelParams -WorksheetName "Processes" 
'Get-Process' | Export-Excel @ExportExcelParams -WorksheetName "Processes" 
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel @ExportExcelParams -WorksheetName "System" 
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel @ExportExcelParams -WorksheetName "Application" 
Get-DbaErrorLog  | Export-Excel @ExportExcelParams -WorksheetName "SQL Error Logs" -Show


## More Information needed

So thats useful but how about if we add a front page which gives some more information?

In [ ]:
$SqlInstance = 'Beard-Desktop'
$Date = Get-Date -Format yyyyMMdd
$Directory = 'C:\temp\xlsx\'
$ExcelFile = $Directory + $Date + '_Incident_For_' + $SqlInstance + '.xlsx'

$ExportExcelParams = @{
    Path =  $ExcelFile 
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
}


Get-Service | Export-Excel @ExportExcelParams -WorksheetName "Services" 
'Get-Process' | Export-Excel @ExportExcelParams -WorksheetName "Processes" 
Get-WinEvent -LogName System -MaxEvents 100  | Export-Excel @ExportExcelParams -WorksheetName "System" 
Get-WinEvent -LogName Application -MaxEvents 100  | Export-Excel @ExportExcelParams -WorksheetName "Application" 
Get-DbaErrorLog  | Export-Excel @ExportExcelParams -WorksheetName "SQL Error Logs" 

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "$Date "
$TitleSheet.Cells["D2"].value = "This Workbook for $sqlinstance"
$TitleSheet.Cells["H3"].value = "Shows the "
$TitleSheet.Cells["H4"].value = "System Event Log,"
$TitleSheet.Cells["H5"].value = "Application Event Log,"
$TitleSheet.Cells["H6"].value = "Services,"
$TitleSheet.Cells["H7"].value = "Processes,"
$TitleSheet.Cells["H8"].value = "and the SQL Error Log."
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

Close-ExcelPackage $excel

Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

                ###############                  
        FileName is C:\temp\xlsx\20221006_Incident_For_Beard-Desktop.xlsx            
                ###############                  



## Charts and Graphs and Pivot Tables

In [ ]:
$Query = "SELECT Top 25000 PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -Database AdventureWorks2017 -Query $Query 
$results | Export-Excel -Path C:\temp\xlsx\query.xlsx -Show

Thats a bit annoying but it is due to the datatable object that is returned. We can fix that using `Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors`

In [ ]:
$Query = "SELECT Top 25000 PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors
$results  | Export-Excel -Path C:\temp\xlsx\newquery.xlsx -Show

Lets format it nicely again

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\formattedquery.xlsx' 
    WorksheetName = 'A Query'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    Show = $true
}
$results | Export-Excel @ExportExcelParams

## Chart

Of course, charts are interesting too.

You can query a table, export the results to a table, and create a chart. We will also remove hte annoying Sheet1 too

In [ ]:
$ExcelFilePath =  'C:\temp\xlsx\chartedquery.xlsx'

$Query = "SELECT  
YEAR(OrdHead.OrderDate) AS Year,
SUM(Details.OrderQty) AS TotalSales
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID
GROUP BY YEAR(OrdHead.OrderDate)
ORDER BY YEAR(OrdHead.OrderDate) 
"
$ExportExcelParams = @{
    Path =  $ExcelFilePath 
    WorksheetName = 'BeardsAreAwesome'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    AutoNameRange = $true
}

$ChartDefinition = @{
    Title = 'BeardyChart' 
    ChartType = 'ColumnClustered' 
    XRange = 'Year'  
    YRange = 'TotalSales'  
    Column = 4
    NoLegend = $true
}

#Gather results
$results = Invoke-DbaQuery -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors

#export results to excel 
$excel = $results | Export-Excel  @ExportExcelParams -PassThru

#create chart defnition
$chart = New-ExcelChartDefinition @ChartDefinition 
# create and save excel
Export-Excel -ExcelPackage $excel -ExcelChartDefinition $chart -WorksheetName 'BeardsAreAwesome' -Show



I'm a DBA, I like doughnuts :-)

In [ ]:
$ExcelFilePath =  'C:\temp\xlsx\DONUTSchartedquery.xlsx'

$Query = "SELECT  
YEAR(OrdHead.OrderDate) AS Year,
SUM(Details.OrderQty) AS TotalSales
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID
GROUP BY YEAR(OrdHead.OrderDate)
ORDER BY YEAR(OrdHead.OrderDate) 
"
$ExportExcelParams = @{
    Path =  $ExcelFilePath 
    WorksheetName = 'BeardsAreAwesome'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    AutoNameRange = $true
}

$ChartDefinition = @{
    Title = 'Beardy Chart of Total Sales By Year as a donut because he LOVES donuts' 
    ChartType = 'Doughnut' 
    XRange = 'Year'  
    YRange = 'TotalSales'  
    Column = 4
    ShowPercent = $true
}

#Gather results
$results = Invoke-DbaQuery -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors

#export results to excel 
$excel = $results | Export-Excel  @ExportExcelParams -PassThru

#create chart defnition
$chart = New-ExcelChartDefinition @ChartDefinition 
# create and save excel
Export-Excel -ExcelPackage $excel -ExcelChartDefinition $chart -WorksheetName 'BeardsAreAwesome' -Show


## Pivot Tables

Pivot tables are very popular with users also.

You can create a pivot table in the same manner

In [ ]:
$ExcelFilePath = 'C:\temp\xlsx\pivotedquery.xlsx' 

$Query = "SELECT PROD.Name, 
OrdHead.OrderDate ,
Details.OrderQty
FROM Sales.SalesOrderDetail AS Details
INNER JOIN Production.Product AS PROD ON Details.ProductID = PROD.ProductID
INNER JOIN Sales.SalesOrderHeader AS OrdHead ON Details.SalesOrderID = OrdHead.SalesOrderID"
$results = Invoke-DbaQuery -Database AdventureWorks2017 -Query $Query  |Select * -ExcludeProperty ItemArray, RowError, RowState, Table, HasErrors

$ExportExcelParams = @{
    Path =  $ExcelFilePath
    WorksheetName = 'BaseQuery'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    Show = $true
    PassThru = $true
}

$PivotTableParams = @{
    PivotTableName = 'BeardyPivot' 
    SourceWorkSheet = 'BaseQuery' 
    PivotData = @{"OrderQty" = "Sum"} 
    PivotRows = 'OrderDate' ,'Name'
}

$excel = $results | Export-Excel @ExportExcelParams 


$PivotTable = New-PivotTableDefinition  @PivotTableParams

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable 
#Remove-Worksheet  -FullName $ExcelFilePath -WorksheetName 'Sheet1' -Show

Wouldnt it be great if we had the "magic" date also

In [ ]:
$ExcelFilePath =  'C:\temp\xlsx\pivoteddatedquery.xlsx'
$ExportExcelParams = @{
    Path = $ExcelFilePath
    WorksheetName = 'BaseQuery'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    Show = $true
    PassThru = $true
}

$PivotTableParams = @{
    PivotTableName = 'BeardyPivot' 
    SourceWorkSheet = 'BaseQuery' 
    PivotData = @{"OrderQty" = "Sum"} 
    PivotRows = 'OrderDate' ,'Name'
    GroupDateRow =  'OrderDate' 
    GroupDatePart = 'Years', 'Months', 'Days' 
    PivotTableStyle = 'Medium1' 
}


$excel = $results | Export-Excel @ExportExcelParams 

$PivotTable = New-PivotTableDefinition @PivotTableParams

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable -Show

and you can add a Pivot Chart also like this

In [ ]:
$ExportExcelParams = @{
    Path =  'C:\temp\xlsx\pivoteddatedchartedquery.xlsx' 
    WorksheetName = 'BaseQuery'
    TableName = 'BeardsAreAwesome'
    TableStyle = 'Dark1'
    AutoSize =  $true
    AutoFilter =  $true
    MoveToStart =  $true
    FreezeTopRow = $true
    Show = $true
    PassThru = $true
}

$PivotTableParams = @{
    PivotTableName = 'BeardyPivot' 
    SourceWorkSheet = 'BaseQuery' 
    PivotData = @{"OrderQty" = "Sum"} 
    PivotRows = 'OrderDate' ,'Name'
    GroupDateRow =  'OrderDate' 
    GroupDatePart = 'Years', 'Months', 'Days' 
    PivotTableStyle = 'Medium1' 
    IncludePivotChart = $true
    ChartTitle = 'BeardyChart' 
    ChartType = 'ColumnClustered' 
    ChartColumn = 6
}


$excel = $results | Export-Excel @ExportExcelParams 

$PivotTable = New-PivotTableDefinition @PivotTableParams 

Export-Excel -ExcelPackage $excel -PivotTableDefinition $PivotTable -Show

## Formatting

Excel enables you to do custom formatting. ImportExcel does the same.

Lets check permissions on an instance and colour code the results depending on the role membership

In [ ]:
$ExcelDirectory = 'c:\temp\xlsx\' # Alter this to the directory you want the file created
$SQlinstance = 'localhost'  # Alter this for the SQL Instance you want to get permissions for

#$SqlInstance = '192.168.2.63,30666'
#$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name beardmi-benadmin-user -AsPlainText), (Get-Secret -Name #beardmi-benadmin-pwd))
#$sql = Connect-DbaInstance -SqlInstance $SqlInstance -SqlCredential $benscreds
#
#$PSDefaultParameterValues = @{
#    "*dba*:SqlInstance" = $sql 
#}

Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_OneTab_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Permissions"

    $excel = Get-DbaUserPermission | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel


Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows :-"
Set-ExcelRange -Worksheet $TitleSheet -Range "B3" -Bold -FontSize 18 
$TitleSheet.Cells["E5"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["E6"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["E7"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["E8"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["E9"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

$TitleSheet.Cells["B11"].Value = "You can filter by Database on the Object column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C11" -FontSize 18

$TitleSheet.Cells["B12"].Value = "You can filter by User/Group/Login on the Member column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C12" -FontSize 18

Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C9" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

Processing localhost
    FileName is c:\temp\xlsx\\localhost_Permssions_OneTab_2022-10-06.xlsx

Finished Processing localhost
                ###############                  
        FileName is c:\temp\xlsx\\localhost_Permssions_OneTab_2022-10-06.xlsx            
                ###############                  

